In [16]:
using Plots
using Dates
using BenchmarkTools

# The Kuramoto-Sivashinsky Equation: Numerics

The Kuramoto-Sivashinsky equation is a PDE that was originally formulated to describe the [fronts of laminar flames](https://www.jstor.org/stable/2100687?seq=1) by G.I. Sivashinsky. The PDE has two forms. In one-dimension, it may be written in *derivative form*:
$$u_t + u u_x + u_{xx} + u_{xxxx} = 0$$
or in *integral form*, which relies on the change of variables $u = v_t$:
$$v_t + \frac 1 2 v_x^2 + v_{xx} + v_{xxxx} = 0$$
In multiple dimensions, it takes the form
$$u_t + \frac 1 2 \lvert \nabla u \rvert^2 + \Delta u + \Delta^2 u = 0$$